# San Andreas Fault seismic analysis

This is a script to download earthquake data from USGS ComCat.
Firstly it downloads the earthquake events within the specific locations bounds and with the magnitude filters.
Then it gets the focal mechanism for each one to determine event type.

Uses the libcomcat python wrappers (https://github.com/usgs/libcomcat)

In [1]:
%matplotlib inline

In [2]:
# stdlib imports
from datetime import datetime
from time import time

# Third party imports
import matplotlib.pyplot as plt
from obspy.geodetics.base import gps2dist_azimuth
import pandas as pd

# Local imports
from libcomcat.dataframes import get_history_data_frame
from libcomcat.search import (get_event_by_id, get_authoritative_info, search)

The first step is to download the general earthquake data from the ComCat database. We set the latitude and longitude bounds of the study area and the minimum and maximum magnitude for analysis.

In [3]:
# latitude and longitude bounds
bounds = [-125.2, -115.312, 32.946, 40.84]

# start/end times for analysis
stime = datetime(2010,1,1)
etime = datetime.utcnow()

# magnitude range
minmag = 4.5
maxmag = 9.9

Then we download a list of SummaryEvent objects for each earthquake. We use the get_authoritative_info() function to retrieve the authoritative magnitudes, locations and depths from each contributing network. Then that information is assembled into a pandas DataFrame to be used for our analysis.

In [8]:
# Retrieve list of events
eventlist = search(starttime=stime,
                  endtime=etime,
                  minlatitude=bounds[2],
                  maxlatitude=bounds[3],
                  minlongitude=bounds[0],
                  maxlongitude=bounds[1],
                  minmagnitude=minmag,
                  maxmagnitude=maxmag,
                  eventtype='earthquake')
print(len(eventlist))

# Retrieve all mag/locations for those events (depending on bandwidth), this can take 30 seconds to a minute per event.
t1 = time()
magdict = {}
columns = []
for event in eventlist:
    mdict, ldict, _ = get_authoritative_info(event.id)
    mdict.update(ldict)
    magdict[event.id] = mdict
    columns += list(mdict.keys())
    
t2 = time()
dt = t2-t1
fmt = 'Elapsed time to retrieve %i magnitudes: %.2f seconds'
print(fmt % (len(eventlist),dt))

# Assemble these dictionaries into a DataFrame
columns = ['eventid'] + list(set(columns))
df = pd.DataFrame(columns=columns)
for eventid, edict in magdict.items():
    edict['eventid'] = eventid
    df = df.append(edict, ignore_index=True)

110
Elapsed time to retrieve 110 magnitudes: 73.99 seconds


In [7]:
# check how many non-null values are in each column
df.count()

eventid         110
nc-longitude     34
nc-mw            31
ci-ml             2
nc-ml             3
ci-mw            37
us-depth         77
us-NA             0
ci-depth         62
us-ml             1
us-mwc            2
ci-mh             9
ci-longitude     62
us-mb             8
nc-depth         34
us-latitude      77
nn-ml            34
nn-latitude      34
ci-latitude      62
us-longitude     77
us-mwb            1
us-mwr           13
nc-latitude      34
nn-depth         34
us-mww           18
nn-longitude     34
ci-mlr           14
dtype: int64